---
# Data Science SE

---
#### **Los objetivos de este notebook son:**

✅ Task 1 → Predictive model

✅ Task 2 → Presentation 


# 1. Introducción:

La UE aporta el 18% del total de las emisiones de gases que provocan el calentamiento global; sin embargo, está cada vez más decidida a tomar la delantera en la lucha contra el cambio climático. Por eso se ha fijado el objetivo de alcanzar las cero emisiones de carbono en 2050.

Para ello, ha puesto en marcha una gran cantidad de recursos que le ayudarán a alcanzar este objetivo en los próximos años, y para ello necesitará su ayuda.


# 2. Conjuntos de datos:

El dataset completo está formado por tres partes:
- `Parte train:`
    - Primera parte: 2 datasets en formato csv. Estos forman el 50% del dataset total.
        - `train_1.csv`
        - `train_2.csv`
    
    - Segunda parte: 3 datasets en formato json. Estos forman el 49% del dataset total.
        - `train_3.json`
        - `train_4.json`
        - `train_5.json` 
    
    - Tercerta parte: 1 dataset en formato pdf. Este forma el 1% del dataset total.

- `Parte test`

# Características de todos los datasets:

- `countryName`: Country in which the facility is located
- `EPRETRSectorCode`: Code of the sector in which the company specialises
- `eptrSectorName`: Name of the sector in which it specialises
- `EPRTRAnnexIMainActivityCode`: Code of the specialisation within the sector in which they operate
- `EPRTRAnnexIMainActivityLabel`: Specialisation within the sector in which they operate
- `FacilityInspireID:` Building identifier
- `facilityName`: Name of the building in which the activity takes place
- `City`: City in which the facility is located
- `CITY ID`: ID to confirm location
- `targetRelease`: Type of polluter to study
- `pollutant`: Type of pollutant emitted (Target variable). In order to follow the same standard, you must encode this variables as follows:
    - Nitrogen oxides (NOX): 0
    - Carbon dioxide (CO2): 1
    - Methane (CH4): 2
- `DAY`: Day on which the report is made
- `MONTH:` Reporting month
- `reportingYear`: Reporting year
- `CONTINENT`: Continent on which the company is located
- `max_wind_speed`: Maximum wind speed
- `avg_wind_speed`: Average wind speed
- `min_wind_speed`: Minimum wind speed
- `max_temp`: Maximum temperature
- `avg_temp`: Average temperature
- `min_temp`: Minimum temperature
- `DAYS WITH FOG`: Total days of the month recorded in the area
- `REPORTER NAME`: Reporter's name

# 3. Leer los datasets y unirlos para formar uno:

Al set tres tipos de datasets hemos de saber unirlos de forma adecuada. Para ello debemos tener en cuenta, las columnas que los conforman, si están mal escritas las columnas...

He tenido un problema con el dataset tipo `pdf` ya que lo he sabido descargar y leer pero juntarlo con las características adecuadas ha sido dificil. Como conforma el 1% del dataset total he optado por dejarlo apartado. No obstante expongo hasta donde he llegado:

- 1. `PDF`: usamos tabula y la API de pdf para poder hacerlo

In [205]:
pip install tabula-py

In [2]:
pip install git+https://github.com/pdftables/python-pdftables-api.git

In [209]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans

from scipy import stats
from scipy.stats import norm

import os
import gc

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.metrics import confusion_matrix

from scipy.stats import zscore
from scipy.stats import iqr

from warnings import filterwarnings
filterwarnings('ignore')

from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

# Import Module
import tabula
import pdftables_api

import warnings
warnings.filterwarnings('ignore')

# setting some globl config
plt.style.use('fivethirtyeight')
cust_color = ['#fdc029',
'#f7c14c',
'#f0c268',
'#e8c381',
'#dfc498',
'#d4c5af',
'#c6c6c6',
'#a6a6a8',
'#86868a',
'#68686d',
'#4b4c52',
'#303138',
'#171820',
]

In [228]:
def eli_plot(df, feature, title):
    
    # Creating a customized chart. and giving in figsize and everything.
    
    fig = plt.figure(constrained_layout=True)
    
    # creating a grid of 3 cols and 3 rows.
    
    grid = gridspec.GridSpec(ncols=3, nrows=2, figure=fig)

    # Customizing the histogram grid.
    
    ax1 = fig.add_subplot(grid[0, :2])
    
    # Set the title.
    
    ax1.set_title('Histogram')
    
    # plot the histogram.
    
    sns.distplot(df.loc[:, feature],
                 hist=True,
                 kde=True,
                 fit=norm,
                  hist_kws={
                 'rwidth': 0.85,
                 'edgecolor': 'black',
                 'linewidth':.5,
                 'alpha': 0.8},
                 ax=ax1,
                 color=cust_color[0])
    
    ax1.axvline(df.loc[:, feature].mean(), color='Green', linestyle='dashed', linewidth=3)

    min_ylim, max_ylim = plt.ylim()
    ax1.text(df.loc[:, feature].mean()*1.25, max_ylim*0.85, 
             'Mean: {:.2f}'.format(df.loc[:, feature].mean()), 
             color='Green', fontsize='12',
             bbox=dict(boxstyle='round',facecolor='red', alpha=0.5))
    ax1.legend(labels=['Actual','Normal'])
    ax1.xaxis.set_major_locator(MaxNLocator(nbins=12))
    
    
    ax1.annotate(
    # Label and coordinate
    'Unexpected Spike here!', xy=(10000, 0.00025),xytext=(10500, 0.0004) ,
    horizontalalignment="center",
    # Custom arrow
    arrowprops=dict(arrowstyle='simple',lw=1, color='black'), fontsize=8
    )

    # customizing the QQ_plot.
    
    ax2 = fig.add_subplot(grid[1, :2])
    
    # Set the title.
    
    ax2.set_title('Probability Plot')
    
    # Plotting the QQ_Plot.
    stats.probplot(df.loc[:, feature],
                   plot=ax2)
    ax2.get_lines()[0].set_markerfacecolor('#e74c3c')
    ax2.get_lines()[0].set_markersize(12.0)
    ax2.xaxis.set_major_locator(MaxNLocator(nbins=16))

    # Customizing the Box Plot:
    
    ax3 = fig.add_subplot(grid[:, 2])
    # Set title.
    
    ax3.set_title('Box Plot')
    
    # Plotting the box plot.
    
    sns.boxplot(y=feature, data=df, ax=ax3, color=cust_color[0])
    ax3.yaxis.set_major_locator(MaxNLocator(nbins=24))

    plt.suptitle(f'{title}', fontsize=24, fontname = 'monospace', weight='bold')
    
    
def count_dist(df, colname=None, fixlabel=False, f_axis=None, fixlabel_n=None, fixlabel_txt=None, max_idx=30, fontsize=12, palette=cust_color, rotation=45,
              title='X distribution', y_label='', shift=-0.005):
    """A function for counting and displaying categorical variables including percentage texts."""
    fig, ax = plt.subplots()
    sns.barplot(y=df[colname].value_counts().index[:max_idx],
                x=df[colname].value_counts().values[:max_idx], palette=palette, 
                edgecolor='black', linewidth=1.5, saturation = 1.5)
    z=df[colname].value_counts().values[:max_idx]
    for n, i in enumerate(df[colname].value_counts().index[:max_idx]):    
        ax.text(df[colname].value_counts().values[:max_idx][n]+shift, 
                n, #Y location
                s=f'{round(z[n]/df.shape[0]*100,1)}%',                 
                va='center', 
                ha='right', 
                color='white', 
                fontsize=fontsize,
                bbox=dict(boxstyle='round',facecolor='black', alpha=0.5))
    if fixlabel:
        if f_axis == 'x':
            labels = [item.get_text() for item in ax.get_xticklabels()]
            labels[fixlabel_n] = fixlabel_txt
            ax.set_xticklabels(labels)
        else:
            labels = [item.get_text() for item in ax.get_yticklabels()]
            labels[fixlabel_n] = fixlabel_txt
            ax.set_yticklabels(labels)            

    plt.title(title, fontname = 'monospace', weight='bold')
    del z
    
    plt.yticks(fontsize=12,rotation=rotation)
    plt.xlabel("Count", fontname = 'monospace', weight='semibold')
    plt.ylabel(y_label, fontname = 'monospace', weight='semibold')
    plt.show()

In [4]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [6]:
# API KEY VERIFICATION
conversion = pdftables_api.Client('frv7t0zf47zt')
  
urls_list = ['pdfs-1.pdf','pdfs81543.pdf']
for urls in urls_list:
    conversion.csv('../input/documentos/train6/'+ str(urls), str(urls)+'.csv')

- 2. `JSON`: Los datasets en formato Json son más fáciles ya que están incorporada en la librería pandas.

In [7]:
URL_3 = 'http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/first'
URL_4 = 'http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/second'
URL_5 = 'http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/third'

In [8]:
df_3 = pd.read_json(URL_3)
df_4 = pd.read_json(URL_4)
df_5 = pd.read_json(URL_5)

- 3. `CSV`: Para ello hemos descargado los datasets desde la web de Schneider.

In [9]:
df_1 = pd.read_csv('../input/csv-format/train1.csv')
df_2 = pd.read_csv('../input/csv-format/train2.csv',sep=";")
test = pd.read_csv('../input/csv-format/test_x.csv')

Ahora vamos a comprobar si hay diferencias entre las columnas de los diferentes datasets. Vemos que entre el `test`y el df_1 (`csv`) hay 3 columnas que no aparecen:
{'EPRTRAnnexIMainActivityCode', 'EPRTRSectorCode', 'test_index'}
Entre los datasets tipo `csv` y `json` hay dos columnas que no aparecen:
{'EPRTRAnnexIMainActivityCode', 'EPRTRSectorCode'}

Partiendo de esta base, vamos a unir todos los datasets. Los pasos son:
- `Paso 1`: ver que columnas son diferentes entre los datasets (comentado anteriormente)

In [10]:
# Todas las columnas de estos 3 dataframe son iguales
set(df_1.columns).difference(df_2.columns)
set(df_2.columns).difference(df_1.columns)
set(test.columns).difference(df_1.columns)

In [11]:
set(df_4.columns).difference(df_3.columns)
set(df_5.columns).difference(df_3.columns)
set(df_4.columns).difference(test.columns)
set(df_5.columns).difference(test.columns)
set(df_3.columns).difference(test.columns)


- `Paso 2`: Vamos a ver las columnas de los dataset tipo json.

In [12]:
df_3.columns = ['id', 'CITY ID', 'CONTINENT', 'City', 'DAY', 'DAY WITH FOGS',
       'EPRTRAnnexIMainActivityCode', 'EPRTRAnnexIMainActivityLabel',
       'EPRTRSectorCode', 'FacilityInspireID', 'MONTH', 'REPORTER NAME',
       'avg_temp', 'avg_wind_speed', 'countryName', 'eprtrSectorName',
       'facilityName', 'max_temp', 'max_wind_speed', 'min_temp',
       'min_wind_speed', 'pollutant', 'reportingYear', 'targetRelease']

df_4.columns = ['id', 'CITY ID', 'CONTINENT', 'City', 'DAY', 'DAY WITH FOGS',
       'EPRTRAnnexIMainActivityCode', 'EPRTRAnnexIMainActivityLabel',
       'EPRTRSectorCode', 'FacilityInspireID', 'MONTH', 'REPORTER NAME',
       'avg_temp', 'avg_wind_speed', 'countryName', 'eprtrSectorName',
       'facilityName', 'max_temp', 'max_wind_speed', 'min_temp',
       'min_wind_speed', 'pollutant', 'reportingYear', 'targetRelease']

df_5.columns = ['id', 'CITY ID', 'CONTINENT', 'City', 'DAY', 'DAY WITH FOGS',
       'EPRTRAnnexIMainActivityCode', 'EPRTRAnnexIMainActivityLabel',
       'EPRTRSectorCode', 'FacilityInspireID', 'MONTH', 'REPORTER NAME',
       'avg_temp', 'avg_wind_speed', 'countryName', 'eprtrSectorName',
       'facilityName', 'max_temp', 'max_wind_speed', 'min_temp',
       'min_wind_speed', 'pollutant', 'reportingYear', 'targetRelease']

In [13]:
df_info_json = pd.DataFrame()

names = df_4.columns
dtype  = ['int64', 'object','object','object','int64','int64','object','object','int64','object','int64',
'object','float64','float64','object','object','object','float64','float64',
'float64','float64','object','int64','object']

df_info_json['Names'] = names
df_info_json['Dtype'] = dtype

df_info_json.head()

- `Paso 3`: Vamos a comprobar el tamaño de todos los datasets

In [14]:
print(df_1.shape, df_2.shape, df_3.shape, df_4.shape, df_5.shape)

- `Paso 4`: Vamos a unir los csv y los json por separado:

In [15]:
# Poner un dataframe debajo del otro:
df_csv = pd.concat([df_1,df_2],axis=0)

# Poner un dataframe debajo del otro:
df_json = pd.concat([df_3,df_4,df_5],axis=0)

In [16]:
set(df_json.columns).difference(df_csv.columns)

In [17]:
df_json_copy = df_json.drop(['EPRTRAnnexIMainActivityCode', 'EPRTRSectorCode', 'id'],axis=1)

- `Paso 5`: Vamos a cambiar el orden de las columnas del dataset para poder combinar los dataset tipo json y tipo csv:

In [19]:
# cambiar orden de las columnas para coincidir con el dataframe df_json_copy
df_csv = df_csv[['CITY ID', 'CONTINENT', 'City', 'DAY', 'DAY WITH FOGS',
       'EPRTRAnnexIMainActivityLabel', 'FacilityInspireID', 'MONTH',
       'REPORTER NAME', 'avg_temp', 'avg_wind_speed', 'countryName',
       'eprtrSectorName', 'facilityName', 'max_temp', 'max_wind_speed',
       'min_temp', 'min_wind_speed', 'pollutant', 'reportingYear',
       'targetRelease']]

- `Paso 6`: Dataset final:

In [20]:
# Poner un dataframe debajo del otro:
df_final = pd.concat([df_csv,df_json_copy],axis=0)

## 3.1 Información general del dataset: 

Vemos que está formado por 65628 entradas y 21 columnas. De las 21 columnas, 11 son de tipo `objeto`. Esto deberemos tenerlo en cuenta para el análisis.

In [21]:
df_final.info()

## 3.2 ¿Hay datos nulos?

Como podemos comprobar no hay datos nulos.

In [22]:
df_final.isnull().sum().sum()

## 3.3 ¿Hay datos duplicados?

Sí hay la presencia de datos duplicados. En concreto el 11.5% de los datos son duplicados. Esto lo podemos ver en la gráfica y la tabla. Posteriormente, borraré estos datos. 

In [27]:
duplicated = pd.DataFrame()
names = ['duplicated values','good dataset']
values = [11.5,88.5]
duplicated['Names'] = names
duplicated['Values %'] = values
duplicated = duplicated.set_index('Names')
duplicated

In [28]:
labels = ['duplicated values','good dataset']
values = [df_final[df_final.duplicated()==True].shape[0],df_final.shape[0]]
colors = ['green','lightgreen','gold', 'mediumturquoise', 'darkorange', 'lightgreen']
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))
fig.update_traces(marker=dict(colors=colors))
fig.show()

In [29]:
df_final = df_final.drop_duplicates()

## 3.5 Reescritura de las columnas: 

Vamos a reescribir las columnas para que todas tengan un formato parecido.

In [30]:
df_final.columns = ['City Id', 'Continent', 'City', 'Day', 'Day of fogs',
       'EPRTRAnnexIMainActivityLabel', 'FacilityInspireID', 'Month',
       'Reporter name', 'Avg_temp', 'Avg_wind_speed', 'CountryName',
       'EprtrSectorName', 'FacilityName', 'Max_temp', 'Max_wind_speed',
       'Min_temp', 'Min_wind_speed', 'Pollutant', 'Year',
       'TargetRelease']

# 4. EDA

## 4.1 Análisis univariable:

Vamos a ver las características de cada variable. En el momento de analizarla, vamos a ver si son significativas para el dataset o no.

### 4.1.1. City Id

Si vemos los datos a continuación, se observa que no es un valor Id como tal, ya que hay datos que se repiten y por otra parte hay demasiados datos poco representados. El mayor representa el 3% del dataset. Por estos motivos, hemos decidido borrarlo del dataset.

In [31]:
df_final['City Id'].value_counts(normalize=True)*100

Hay demasiados valores con poco peso en las variables, por lo tanto eliminamos esta columna

In [32]:
df_final = df_final.drop(['City Id'],axis=1)

### 4.1.2. Continent

Todo la columna indica que el dataset es de Europa. Es lógico, ya que la investigación es de Europa. Por tanto, debido que no aporta mayor información, la eliminaremos.

In [33]:
df_final['Continent'].unique()

In [34]:
df_final = df_final.drop(['Continent'],axis=1)

### 4.1.3. City

Aquí miraremos las ciudades que han participado en este estudio. Comprobaremos que hay más de 1500 datos que no expone la ciudad. Para tener una mayor comprensión de estos datos, hemos mirado el `CountryName` y hemos visto que es de `United Kingdom`. Por tanto, hemos reemplazado los datos `--` a `Other_UK`.

In [35]:
df_final[df_final['City']=='--']['CountryName'].unique() # United Kingdom
# reemplazamos por London

In [36]:
df_final.City.replace('--','Other_UK', inplace=True)

A continuación veremos un gráfico de las ciudades que más representan este dataset. La ciudad que más destaca es `Other_UK` seguidamente de `Antwerpen` y `Duisburg`.

In [107]:
count_dist(df=df_final, colname='City', 
           max_idx=12, fontsize=16, rotation=0,
           title='City Distribution\n',
           shift=400, y_label = 'City')

### 4.1.4 Day

Vemos que se recopila datos desdel 1-28 de cada més incluidos. Esto debe ser debido al mes de Febrero. 

In [38]:
df_final['Day'].sort_values().unique() # hay hasta 28 días

Si vemos la gráfica a continuación sobre su distribución, vemos en qq-plot que no siguen una distribución normal. La media de los datos está en el día 14.

In [106]:
eli_plot(df_final, 'Day', 'Day Distribution\n')

### 4.1.5. Day of fogs

Los días de viento como máximo representan 19 días de 28. Pero normalmente, la mediana indica 1 día.

In [39]:
df_final['Day of fogs'].sort_values().unique() # hasta 19 días con viento

In [105]:
eli_plot(df_final, 'Day of fogs', 'Day of fogs Distribution\n')

###  4.1.6. EPRTRAnnexIMainActivityLabel


Indica la actividad de la producción de polución. Vemos que las 3 más destacadas son:
- Thermal power stations and other combustion installations.
- Landfills (excluding landfills of inert waste and landfills, which were definitely closed before 16.7.2001 or for which the after-care phase required by the competent authorities according to Article 13 of Council Directive 1999/31/EC of 26 April 1999 on the landfill of waste has expired)
- Installations for the incineration of non-hazardous waste in the scope of Directive 2000/76/EC of the European Parliament and of the Council of 4 December 2000 on the incineration of waste

In [104]:
count_dist(df=df_final, colname='EPRTRAnnexIMainActivityLabel', 
           max_idx=12, fontsize=16, rotation=0,
           title='Activity label Distribution\n',
           shift=400, y_label = 'Activity label')

### 4.1.7. FacilityInspireID

En esta característica, vemos una serie de número sobre la identificación del edificio donde se hace el estudio. Como hay poco peso en la representabilidad de los datos, dicidimos eliminarlo.

In [42]:
df_final.FacilityInspireID.value_counts()

In [43]:
df_final = df_final.drop(['FacilityInspireID'],axis=1)

### 4.1.8. Reporter name

Aquí vemos una lista del nombre de los reporter. Hemos querido jugar un poco con la variable y ver en qué paises se han centrado más. Los 5 reporters más importantes son: 

In [45]:
df_report_names = pd.DataFrame()
report_names = ['Michael Brown','James Smith','Michael Smith','Michael Williams','Robert Jones']
value_reports = [25,25,23,22,22]
df_report_names['Report names'] = report_names
df_report_names['Count'] = value_reports
df_report_names

Podemos ver que:
- Michael Brown se centra en `España`.
- James Smith se centra en `Polonia`.
- Michael Smith se centra en `Italia`
- Michael Williams se centra en `España`.
- Robert Jones se centra en `UK` .

Hemos decidido borrar esta característica ya que no aporta información útil al conjunto del dataset.

In [47]:
df_mb = df_final[df_final['Reporter name']=='Michael Brown']
df_js = df_final[df_final['Reporter name']=='James Smith']
df_ms = df_final[df_final['Reporter name']=='Michael Smith']
df_mw = df_final[df_final['Reporter name']=='Michael Williams']
df_rj = df_final[df_final['Reporter name']=='Robert Jones']
df_report_info = pd.concat([df_mb,df_js,df_ms,df_mw,df_rj],axis=0)

In [49]:
fig = px.sunburst(data_frame=df_report_info,
                  path=['Reporter name', 'CountryName'],
                  color='CountryName',
                  color_discrete_sequence=cust_color[::-4],
                  title='Top Reporter Names vs Country '
                 )

fig.update_traces(textinfo='label')
fig.update_layout(margin=dict(t=40, l=0, r=0, b=0))
fig.show()

In [50]:
df_final = df_final.drop(['Reporter name'],axis=1)

### 4.1.9. Average temperature

A continuación veremos la distribució de esta variable. Se puede observar que no sigue una distribución normal y que la mediana está a 11 grados.

In [70]:
eli_plot(df_final, 'Avg_temp', 'Average Temperature Distribution\n')

### 4.1.10. Average wind speed

Vemos a continuación, que la característica no sigue una distribución normal y que la media está en 18.01 unidades de viento.

In [72]:
eli_plot(df_final, 'Avg_wind_speed', 'Average wind speed Distribution\n')

### 4.1.11. CountryName

A los tres paises que se le han realizado más estudios son: UK, Germany y France.

In [87]:
df_final.CountryName.value_counts() 
fig, ax = plt.subplots(1, 2, figsize=(12,8))
sns.barplot(x=df_final.CountryName.value_counts()[:12].index, 
            y=df_final.CountryName.value_counts()[:12].values, 
            palette=cust_color, ax=ax[0],
           edgecolor='black', linewidth=1.5, saturation=1.5)
ax[0].yaxis.set_major_locator(MaxNLocator(nbins=20))
ax[0].tick_params(axis='x', which='major', labelsize=10, rotation=90)
ax[0].set_ylabel('Count', weight='semibold', fontname = 'monospace', rotation=90)
ax[0].set_xlabel('Country Group', weight='semibold', fontname = 'monospace')

ax[1].pie(df_final.CountryName.value_counts()[:12], 
          labels = df_final.CountryName.value_counts()[:12].index, 
          colors = cust_color, autopct='%1.1f%%',
        explode=[0.03 for i in df_final.CountryName.value_counts()[:12].index])


plt.suptitle('Country Distribution', fontname = 'monospace', weight='bold')
plt.show()

### 4.1.12.Sector in which it specialises

Hay 9 tipos de sectores. La más representada es `Energy Sector` seguida de `Waste and wastewater management`

In [102]:
count_dist(df=df_final, colname='EprtrSectorName', 
           max_idx=12, fontsize=16, rotation=0,
           title='Sector Name Distribution\n',
           shift=400, y_label = 'Sector Name')

### 4.1.13.Facility Name

Esta característica representa el lugar donde se han realizado los estudios. Es una característica poco significativa ya que la que representa mayor peso es `Enel Produzione S.p.A` con un 0.35% del dataset total.

In [75]:
(df_final.FacilityName.value_counts(normalize=True)*100)[:9]
# es una variable poco significativa debido a que hay muchos valores unitarios y el mayor
# presenta una cuota del 0.35 % del total.

In [76]:
df_final = df_final.drop(['FacilityName'],axis=1)

### 4.1.14. Max temperature

Representa la máxima temperatura de todo el dataset. Vemos que se aproxima a una distribución Gaussiana. 

In [77]:
eli_plot(df_final, 'Max_temp', 'Max temperature Distribution\n')

### 4.1.15. Max wind speed

Representa la velocidad máxima del viento. Su distribución sí recuerda a una distribución normal con una media de 15.52 unidades de viento.

In [78]:
eli_plot(df_final, 'Max_wind_speed', 'Max wind speed Distribution\n')

### 4.1.16. Min temperature

Representa la distribución de la temperatura mínima del dataset. Vemos que recuerda a una distribución normal con una media de 13.45 grados.

In [79]:
eli_plot(df_final, 'Min_temp', 'Min temperature Distribution\n')

### 4.1.17. Min wind speed

Representa la distribución de la velocidad mínima de viento. Recuerda a una distribución Gaussiana con una media de 22.52 unidades de viento.

In [80]:
eli_plot(df_final, 'Min_wind_speed', 'Min wind speed Distribution\n')

### 4.1.18. Pollutant

Esta característica es el target del dataset. Está formada por tres variables, por lo que es una variable categórica. El dataset no se puede considerar desbalanceado ya que todas las categorías están adecuadamente representadas.

In [81]:
df_final.Pollutant.value_counts()

In [92]:
# plotting and styling
fig, ax = plt.subplots(figsize=(12,4))
sns.barplot(x=df_final_copy['Pollutant'].value_counts().index,
            y=df_final_copy['Pollutant'].value_counts().values,
            palette=cust_color[::4],
            edgecolor='black', linewidth=1.5, saturation=1.5)
plt.xlabel("Type of Pollutant", fontname = 'monospace', weight='semibold')
plt.ylabel("Count", fontname = 'monospace', weight='semibold')
plt.title('Pollutant Distribution', fontname = 'monospace', weight='bold');

### 4.1.19. Reporting Year

Este es el año que se hizo el estudio. Los años son desde el 2008 - 2020 incluidos ambos. No obstante, vemos que hay más representación en los años iniciales que en los finales.

In [111]:
df_final.Year.value_counts() 
fig, ax = plt.subplots(1, 2, figsize=(12,8))
sns.barplot(x=df_final.Year.value_counts()[:12].index, 
            y=df_final.Year.value_counts()[:12].values, 
            palette=cust_color, ax=ax[0],
           edgecolor='black', linewidth=1.5, saturation=1.5)
ax[0].yaxis.set_major_locator(MaxNLocator(nbins=20))
ax[0].tick_params(axis='x', which='major', labelsize=10, rotation=90)
ax[0].set_ylabel('Count', weight='semibold', fontname = 'monospace', rotation=90)
ax[0].set_xlabel('Year Group', weight='semibold', fontname = 'monospace')

ax[1].pie(df_final.Year.value_counts()[:12], 
          labels = df_final.Year.value_counts()[:12].index, 
          colors = cust_color, autopct='%1.1f%%',
        explode=[0.03 for i in df_final.Year.value_counts()[:12].index])


plt.suptitle('Year Distribution', fontname = 'monospace', weight='bold')
plt.show()

### 4.1.20.Target Release

Esta columna solo tiene un elemento `Air` por lo que decidimos eliminarla ya que no aporta información adicional al dataset.

In [83]:
df_final.TargetRelease.value_counts()
# como esta columna solo tiene un valor 'AIR' lo podemos eliminar

In [84]:
df_final = df_final.drop(['TargetRelease'],axis=1)

## 4.2 Dataset Final

El dataset final consta de todo lo que hemos mencionado y a continuación añadiremos una gráfica de tiempo para comprobar que el dataset representa adecuadamente todos los tiempos y no hay desbalance en los datos. 

In [85]:
df_final_copy = df_final.copy()

In [91]:
fig, ax = plt.subplots(1, 3, figsize=(12,8))
sns.barplot(x=df_final_copy.Year.value_counts().index, 
            y=df_final_copy.Year.value_counts().values, 
            palette=cust_color, ax=ax[0],
           edgecolor='black', linewidth=1.5, saturation=1.5)
ax[0].yaxis.set_major_locator(MaxNLocator(nbins=20))
ax[0].tick_params(axis='x', which='major', labelsize=10, rotation=90)
ax[0].set_ylabel('Count', weight='semibold', fontname = 'monospace', rotation=90)
ax[0].set_xlabel('Year Distribution', weight='semibold', fontname = 'monospace')

ax[1].pie(df_final_copy.Month.value_counts(), 
          labels = df_final_copy.Month.value_counts().index, 
          colors = cust_color, autopct='%1.1f%%',
        explode=[0.03 for i in df_final_copy.Month.value_counts().index])
ax[1].set_xlabel('Month Distribution', weight='semibold', fontname = 'monospace')

ax[2].pie(df_final_copy.Day.value_counts(), 
          labels = df_final_copy.Day.value_counts().index, 
          colors = cust_color, autopct='%1.1f%%',
        explode=[0.03 for i in df_final_copy.Day.value_counts().index])
ax[2].set_xlabel('Day Distribution', weight='semibold', fontname = 'monospace')


plt.suptitle('Time Distribution', fontname = 'monospace', weight='bold')
plt.show()

## 4.2 Analisis bi-variable

Vamos a ver algunas relaciones interesantes para tener una idea más específica del dataset. 

### 4.2.1 Country vs City:

A continuación podemos ver qué ciudades son representadas mayormente de cada país. Por ejemplo: La ciudad a la que se le ha hecho más estudios de Alemania es `Duisburg`.

In [90]:
fig = px.sunburst(data_frame=df_country,
                  path=['CountryName', 'City'],
                  color='City',
                  color_discrete_sequence=cust_color[::-4],
                  title='12 top Country vs City '
                 )

fig.update_traces(textinfo='label')
fig.update_layout(margin=dict(t=40, l=0, r=0, b=0))
fig.show()

### 4.2.2 ¿Qué sector es el que contribuye más a la contaminación según el Country?

Como podemos ver `Energy sector` contribuye en primer lugar a UK, después de Alemania e Italia. En cambio la `Industria minera`contribuye más a Italia que Alemania.

In [115]:
fig = px.sunburst(data_frame=df_final_copy,
                  path=['EprtrSectorName', 'CountryName'],
                  color='CountryName',
                  color_discrete_sequence=cust_color[::-4],
                  title='Sector Name vs Country '
                 )

fig.update_traces(textinfo='label')
fig.update_layout(margin=dict(t=40, l=0, r=0, b=0))
fig.show()

### 4.2.3 ¿Dónde hay más días de viento según Country?

En `UK` hay 11 días de viento mientras que en Alemania y Francia hay 2 y 1 respectivamente. 

In [121]:
fig = px.sunburst(data_frame=df_final_copy,
                  path=['CountryName', 'Day of fogs'],
                  color='Day of fogs',
                  color_discrete_sequence=cust_color[::-4],
                  title='12 top Country vs City '
                 )

fig.update_traces(textinfo='label')
fig.update_layout(margin=dict(t=40, l=0, r=0, b=0))
fig.show()

In [148]:
df_final_copy_cont = df_final_copy.select_dtypes(include=['float64','int64'])

## 4.3 Análisis del target

En el análisis univariable ya hemos visto la distribución del target. Ahora veremos cómo se relacionada con otras variables.

Vemos que Nitrogen oxides y Carbon dioxide están más presentes en `Alemania` mientras que Methane está más presente en `UK`.

In [117]:
fig = px.sunburst(data_frame=df_final_copy,
                  path=['Pollutant', 'CountryName'],
                  color='CountryName',
                  color_discrete_sequence=cust_color[::-4],
                  title='Pollutant vs Country '
                 )

fig.update_traces(textinfo='label')
fig.update_layout(margin=dict(t=40, l=0, r=0, b=0))
fig.show()

A continuación, analizaremos la relación entre Pollutant y Year. Gracias a ello podemos ver, que los años finales 2018-2019-2020 hay menos contaminación en la atmosfera por estos gases que en los primeros años. Esto es debido a las políticas empleadas para su disminución.

In [119]:
fig = px.sunburst(data_frame=df_final_copy,
                  path=['Pollutant', 'Year'],
                  color='Year',
                  color_discrete_sequence=cust_color[::-4],
                  title='Pollutant vs Year'
                 )

fig.update_traces(textinfo='label')
fig.update_layout(margin=dict(t=40, l=0, r=0, b=0))
fig.show()

Seguidamente, veremos cuál es la actividad que genera más polución a la atmosfera. Como podemos ver las `instalaciones de carbono` son las que más contribuyen a la emisión de estos gases. 

In [120]:
fig = px.sunburst(data_frame=df_final_copy,
                  path=['Pollutant', 'EPRTRAnnexIMainActivityLabel'],
                  color='EPRTRAnnexIMainActivityLabel',
                  color_discrete_sequence=cust_color[::-4],
                  title='¿Qué actividad provoca más polución? '
                 )

fig.update_traces(textinfo='label')
fig.update_layout(margin=dict(t=40, l=0, r=0, b=0))
fig.show()

# 5. Correcciones del test

El test es el dataset en bruto. No le aplicaremos casi ninguna transformación pero sí quiero ordenar las columnas y eliminar esas que no son significativas según nuestro criterio.

In [161]:
test = test[['test_index','CITY ID', 'CONTINENT', 'City', 'DAY', 'DAY WITH FOGS',
       'EPRTRAnnexIMainActivityLabel', 'FacilityInspireID', 'MONTH',
       'REPORTER NAME', 'avg_temp', 'avg_wind_speed', 'countryName',
       'eprtrSectorName', 'facilityName', 'max_temp', 'max_wind_speed',
       'min_temp', 'min_wind_speed', 'reportingYear',
       'targetRelease']]

In [163]:
test.columns =  ['test_index','City Id', 'Continent', 'City', 'Day', 'Day of fogs',
       'EPRTRAnnexIMainActivityLabel', 'FacilityInspireID', 'Month',
       'Reporter name', 'Avg_temp', 'Avg_wind_speed', 'CountryName',
       'EprtrSectorName', 'FacilityName', 'Max_temp', 'Max_wind_speed',
       'Min_temp', 'Min_wind_speed', 'Year',
       'TargetRelease']

In [164]:
test = test.drop(['City Id','Continent','FacilityInspireID','Reporter name','FacilityName','TargetRelease'],axis=1)

# 6. Algoritmo

Ahora vamos a contruir nuestro algoritmo. Primero vamos a transformar las columnas tipo `object` en formato numérico no ordinal. Para ello usamos la librería LabelEnconder. Como vemos a continuación, antes que nada, comprobamos si hay valores nulos y sus características generales.

In [128]:
df_final_copy.info()

Vamos a separar el `train` en `X` e `y`, y vamos a dividir según datos tipo `object`  o no. Seguidamente le aplicaremos a los datos `object` el LabelEncoder.

In [ ]:
y = df_final_copy['Pollutant']
X = df_final_copy.drop(['Pollutant'], axis=1)

In [ ]:
X_cat = X.select_dtypes(include='object')
X_cont = X.select_dtypes(include=['int64','float64'])

In [135]:
le = LabelEncoder()

In [143]:
X_cat = X_cat.apply(le.fit_transform)

Ahora uniremos los datos tipo `object` y los otros en un nuevo dataframe numérico.

In [144]:
X_final = pd.concat([X_cont, X_cat],axis=1)

Ahora, vamos a hacer lo mismo para el `test`.

In [170]:
test_cat = test.select_dtypes(include='object')
test_cont = test.select_dtypes(exclude='object')

In [171]:
test_cat_enc=test_cat.apply(le.fit_transform)

In [174]:
test_final = pd.concat([test_cont,test_cat_enc],axis=1)

## 5.1 Selección de características

Quiero realizar este apartado para ver que características, según el modelo mutual information, son más relevantes. Si nos fijamos son dice que las características:
'Year', 'City','EPRTRAnnexIMainActivityLabel' y 'CountryName' son las más relevantes.

In [180]:
from matplotlib import pyplot

x_train_split, x_test_split, y_train_split, y_test_split = train_test_split(X_final, y, test_size=0.33, random_state=2022)

# feature selection
def select_features_mutual(X_train, y_train, X_test):
    # configure to select all features
    fs = SelectKBest(score_func=mutual_info_classif, k='all')
    # learn relationship from training data
    fs.fit(X_train, y_train)
    # transform train input data
    X_train_fs = fs.transform(X_train)
    # transform test input data
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

# feature selection
X_train_mutual, X_test_mutal, mutual = select_features_mutual(x_train_split, y_train_split, x_test_split)


# what are scores for the features
for i in range(len(mutual.scores_)):
    print('Feature %d: %f' % (i, mutual.scores_[i]))
    # plot the scores
pyplot.title('Mutual info classification feature selection\n', fontdict=None, loc='center')
pyplot.bar([i for i in range(len(mutual.scores_))], mutual.scores_)
pyplot.show()

## 5.2 Creación del modelo:

Hemos escogido un modelo donde las característica que tienen poco peso pasen a ser más significativas para no perder información. Esto lo hacemos gracias al algoritmo de Deep Learning Gated Residual Network y Variable Selection. 
- `Paso 1`: Determinar las variables del dataset para entrenarlo

In [194]:
# split dataframes for later modeling
X = X_final.copy()
y = y.copy()

X_test_id = test_final.copy() # este tiene el id
X_test = X_test_id.drop(['test_index'],axis=1)

- `Paso 2`: Comprobar el tamaño del dataset:

In [195]:
print(X.shape, y.shape, X_test.shape)

- `Paso 3`: Aplicar encoding al target con keras.io

In [196]:
target = keras.utils.to_categorical(le.fit_transform(y))

- `Paso 4`: Comprobar una vez más el tamaño del dataset

In [197]:
gc.collect()
print(X.shape, y.shape, target.shape, X_test.shape)

- `Paso 5`: Definir las funciones de ayuda de nuestro dataset:
    - set_seed: para determinar el seed
    - plot_eval_results: para ilustrar los gráficos sobre las funciones de pérdida según el cv escogido.
    - plot_cv: para visualizar las métricas de nuestro dataset. Recordamos que la métrica que nos importa es `f1-score`.

In [198]:
# define helper functions
def set_seed(seed):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    print(f"Seed set to: {seed}")

def plot_eval_results(scores, n_splits):
    cols = 10
    rows = int(np.ceil(n_splits/cols))
    
    fig, ax = plt.subplots(rows, cols, tight_layout=True, figsize=(20,2.5))
    ax = ax.flatten()

    for fold in range(len(scores)):
        df_eval = pd.DataFrame({'train_loss': scores[fold]['loss'], 'valid_loss': scores[fold]['val_loss']})

        sns.lineplot(
            x=df_eval.index,
            y=df_eval['train_loss'],
            label='train_loss',
            ax=ax[fold]
        )

        sns.lineplot(
            x=df_eval.index,
            y=df_eval['valid_loss'],
            label='valid_loss',
            ax=ax[fold]
        )

        ax[fold].set_ylabel('')

    sns.despine()

def plot_cm(cm):
    metrics = {
        'accuracy': cm / cm.sum(),
        'recall' : cm / cm.sum(axis=1),
        'precision': cm / cm.sum(axis=0)
    }
    
    fig, ax = plt.subplots(1,3, tight_layout=True, figsize=(15,5))
    ax = ax.flatten()

    mask = (np.eye(cm.shape[0]) == 0) * 1

    for idx, (name, matrix) in enumerate(metrics.items()):

        ax[idx].set_title(name)

        sns.heatmap(
            data=matrix,
            cmap=sns.dark_palette("#69d", reverse=True, as_cmap=True),
            cbar=False,
            mask=mask,
            lw=0.25,
            annot=True,
            fmt='.2f',
            ax=ax[idx]
        )
    sns.despine()

- `Paso 6`: Vamos a difinir dos callbacks: ReduceLROnPlateau y EarlyStopping donde le hemos aplicado el loss de `f1-score`.

In [199]:
# define callbacks
lr = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", 
    factor=0.5, 
    patience=5, 
    verbose=True
)

es = keras.callbacks.EarlyStopping(
    monitor="val_get_f1", 
    patience=10, 
    verbose=True, 
    mode="max", 
    restore_best_weights=True
)

- `Paso 7`: contruir nuestro modelo integro teniendo en cuenta la métrica requerida.

In [200]:
class GatedLinearUnit(layers.Layer):
    def __init__(self, units):
        super(GatedLinearUnit, self).__init__()
        self.linear = layers.Dense(units)
        self.sigmoid = layers.Dense(units, activation="sigmoid")

    def call(self, inputs):
        return self.linear(inputs) * self.sigmoid(inputs)

class GatedResidualNetwork(layers.Layer):
    def __init__(self, units, dropout_rate):
        super(GatedResidualNetwork, self).__init__()
        self.units = units
        self.elu_dense = layers.Dense(units, activation="elu")
        self.linear_dense = layers.Dense(units)
        self.dropout = layers.Dropout(dropout_rate)
        self.gated_linear_unit = GatedLinearUnit(units)
        self.layer_norm = layers.LayerNormalization()
        self.project = layers.Dense(units)

    def call(self, inputs):
        x = self.elu_dense(inputs)
        x = self.linear_dense(x)
        x = self.dropout(x)
        if inputs.shape[-1] != self.units:
            inputs = self.project(inputs)
        x = inputs + self.gated_linear_unit(x)
        x = self.layer_norm(x)
        return x

class VariableSelection(layers.Layer):
    def __init__(self, num_features, units, dropout_rate):
        super(VariableSelection, self).__init__()
        self.grns = list()
        for idx in range(num_features):
            grn = GatedResidualNetwork(units, dropout_rate)
            self.grns.append(grn)
        self.grn_concat = GatedResidualNetwork(units, dropout_rate)
        self.softmax = layers.Dense(units=num_features, activation="softmax")

    def call(self, inputs):
        v = layers.concatenate(inputs)
        v = self.grn_concat(v)
        v = tf.expand_dims(self.softmax(v), axis=-1)

        x = []
        for idx, input in enumerate(inputs):
            x.append(self.grns[idx](input))
        x = tf.stack(x, axis=1)

        outputs = tf.squeeze(tf.matmul(v, x, transpose_a=True), axis=1)
        return outputs

- `Paso 8`: Contrucción de los inputs y el encode.

In [201]:
def create_model_inputs():
    inputs = {}
    for feature_name in X.columns:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(), dtype=tf.float32
        )
    return inputs

def encode_inputs(inputs, encoding_size):
    encoded_features = []
    for col in range(inputs.shape[1]):
        encoded_feature = tf.expand_dims(inputs[:, col], -1)
        encoded_feature = layers.Dense(units=encoding_size)(encoded_feature)
        encoded_features.append(encoded_feature)
    return encoded_features

def create_model(encoding_size, dropout_rate=0.15):
    inputs = layers.Input(len(X.columns))
    feature_list = encode_inputs(inputs, encoding_size)
    num_features = len(feature_list)

    features = VariableSelection(num_features, encoding_size, dropout_rate)(
        feature_list
    )

    outputs = layers.Dense(units=target.shape[-1], activation="softmax")(features)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

- `Paso 9`: Remarcar las excepciones del modelo.

In [202]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    tf_strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print("Running on TPU:", tpu.master())
except:
    tf_strategy = tf.distribute.get_strategy()
    print(f"Running on {tf_strategy.num_replicas_in_sync} replicas")
    print("Number of GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

- `Paso 10`: crear la función de la métrica que el problema nos demanda.

In [203]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

- `Paso 11`: importar todo el modelo para entrenarlo

In [204]:
seed = 2022
set_seed(seed)

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

predictions = []
oof_preds = {'y_valid': list(), 'y_hat': list()}
scores_nn = {fold:None for fold in range(cv.n_splits)}

for fold, (idx_train, idx_valid) in enumerate(cv.split(X,y)):
    X_train, y_train = X.iloc[idx_train], target[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], target[idx_valid]
    
    scl = RobustScaler()
    X_train = scl.fit_transform(X_train)
    X_valid = scl.transform(X_valid)
    
    with tf_strategy.scope():
        model = create_model(encoding_size=128)

        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=1e-3),
            loss=keras.losses.CategoricalCrossentropy(),
            metrics=[get_f1]
        )
        
    history = model.fit(
        X_train, y_train,
        validation_data=(X_valid, y_valid),
        epochs=90,
        batch_size=4096,
        shuffle=True,
        verbose=False,
        callbacks=[lr,es]
    )
    
    scores_nn[fold] = history.history
    
    oof_preds['y_valid'].extend(y.iloc[idx_valid])
    oof_preds['y_hat'].extend(model.predict(X_valid, batch_size=4096))
    
    prediction = model.predict(scl.transform(X_test), batch_size=4096) 
    predictions.append(prediction)
    
    #del model, prediction
    gc.collect()
    K.clear_session()
    
    print('_'*65)
    print(f"Fold {fold+1} || Min Val Loss: {np.min(scores_nn[fold]['val_loss'])}")
    print('_'*65)
    
print('_'*65)
overall_score = [np.min(scores_nn[fold]['val_loss']) for fold in range(cv.n_splits)]
print(f"Overall Mean Validation Loss: {np.mean(overall_score)}")

## 5.3 Evaluación del modelo:

In [210]:
#plot_eval_results(scores_nn, cv.n_splits)

## 5.4 Observación de las métricas

In [212]:
# prepare oof_predictions
oof_y_true = np.array(oof_preds['y_valid'])
oof_y_hat = le.inverse_transform(np.argmax(oof_preds['y_hat'], axis=1))

# create confusion matrix, calculate accuracy, recall & precision
cm = pd.DataFrame(data=confusion_matrix(oof_y_true, oof_y_hat, labels=le.classes_), index=le.classes_, columns=le.classes_)

In [214]:
cm = confusion_matrix(oof_y_true, oof_y_hat)
ix = np.arange(cm.shape[0])
cm[ix, ix] = 0
col_names = [f'Polucion={cls}' for cls in le.classes_]
cm = pd.DataFrame(cm, columns=col_names, index=col_names)
sns.heatmap(cm, cmap='Blues', annot=True, fmt='d').set(title=f'{cm.sum().sum()} misses\n ');

In [215]:
cm = confusion_matrix(oof_y_true, oof_y_hat)
ix = np.arange(cm.shape[0])
col_names = [f'Polución={cls}' for cls in le.classes_]
cm = pd.DataFrame(cm, columns=col_names, index=col_names)
sns.heatmap(cm, cmap='Blues', annot=True, fmt='d').set(title=f'Confusion matrix\n');

Como podemos observar, hay muchos datos erróneamente clasificados. Esto es un problema que se debería solventar. Ahora vamos a ver el reporte:

In [216]:
from sklearn.metrics import classification_report

In [217]:
print(classification_report(oof_y_true, oof_y_hat))

Vemos que `f1-score (macro)` es del 0.61. Es bastante bajo. Se debería replantear algunos criterios para intentar mejorarlo.

## 5.4 Resultados finales

In [218]:
#create final prediction, inverse labels to original classes
final_predictions = le.inverse_transform(np.argmax(sum(predictions), axis=1))

In [222]:
#submission
submission = X_test_id.drop(['Day', 'Day of fogs', 'Month', 'Avg_temp',
       'Avg_wind_speed', 'Max_temp', 'Max_wind_speed', 'Min_temp',
       'Min_wind_speed', 'Year', 'City', 'EPRTRAnnexIMainActivityLabel',
       'CountryName', 'EprtrSectorName'], axis=1)
submission['Prediction'] = final_predictions
submission.head()

In [225]:
submission.to_csv('predictions.csv', index=False)
submission.to_json('predictions.json')

Como se puede ver, la mayoría de predicciones de los datos `test` han sido NOX.

In [227]:
fig, axes = plt.subplots(1, 1, figsize=(18, 10))
fig.suptitle('\nNúmero de predicciones para las diferentes categorías de Polución\n\n', size=25)
sns.countplot(final_predictions,palette = "Blues")
sns.despine()